# LETS DO IT BOYSSSSSS

In [84]:
import gzip
import json
import random
import numpy as np
import pandas as pd

In [85]:
def reading_gz(filename):
    f = gzip.open(filename, 'rt', encoding="utf8")
    lines = []
    for line in f:
        liness = json.loads(line.strip())
        lines.append(liness)
    return lines

In [86]:
data_modcloth = reading_gz('modcloth_final_data.json.gz')
data_renttherunway = reading_gz('renttherunway_final_data.json.gz')

The data_modcloth has 82790 data and the data_renttherunway has 192544 items

In [87]:
len(data_modcloth),len(data_renttherunway)

(82790, 192544)

In [88]:
data_tb = pd.DataFrame(data_renttherunway)

In [89]:
data_tb

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,fit,420272,34d,2260466,137lbs,10,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,"5' 8""",14,28,"April 20, 2016"
1,fit,273551,34b,153475,132lbs,10,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,"5' 6""",12,36,"June 18, 2013"
2,fit,360448,NaN,1063761,NaN,10,party,This hugged in all the right places! It was a ...,NaN,It was a great time to celebrate the (almost) ...,sheath,"5' 4""",4,116,"December 14, 2015"
3,fit,909926,34c,126335,135lbs,8,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,"5' 5""",8,34,"February 12, 2014"
4,fit,151944,34b,616682,145lbs,10,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,"5' 9""",12,27,"September 26, 2016"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192539,fit,66386,34dd,2252812,140lbs,10,work,Fit like a glove!,hourglass,LOVE IT!!! First Item Im thinking of buying!,jumpsuit,"5' 9""",8,42,"May 18, 2016"
192540,fit,118398,32c,682043,100lbs,10,work,The pattern contrast on this dress is really s...,petite,LOVE it!,dress,"5' 1""",4,29,"September 30, 2016"
192541,fit,47002,36a,683251,135lbs,6,everyday,"Like the other DVF wraps, the fit on this is f...",straight & narrow,"Loud patterning, flattering fit",dress,"5' 8""",8,31,"March 4, 2016"
192542,fit,961120,36c,126335,165lbs,10,wedding,This dress was PERFECTION. it looked incredib...,pear,loved this dress it was comfortable and photog...,dress,"5' 6""",16,31,"November 25, 2015"


#  data_renttherunway

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy
import urllib
import scipy.optimize
import random
from collections import defaultdict # Dictionaries with default values
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
import ast
import numpy as np

In [91]:
np.random.shuffle(data_renttherunway)

In [92]:
num_90th = int(len(data_renttherunway)*0.9)
num_90th

173289

In [93]:
train_set = data_renttherunway[0:num_90th]
validation_set = data_renttherunway[num_90th:]

## updating the category 

In [94]:
categorys= np.unique([w['category'] for w in data_renttherunway])
categoryId = dict(zip(categorys, 
                  range(len(categorys))))

In [95]:
categorys.shape

(68,)

There are some repeated meaning category, so we need to stem the category

In [96]:
stemmer = PorterStemmer()

In [97]:
category_stem= np.unique([stemmer.stem(w['category'])for w in data_renttherunway])
categorystemId = dict(zip(categorys, 
                  range(len(categorys))))

In [98]:
category_stem

array(['ballgown', 'blazer', 'blous', 'blouson', 'bomber', 'buttondown',
       'caftan', 'cami', 'cape', 'cardigan', 'coat', 'combo', 'crewneck',
       'culott', 'down', 'dress', 'duster', 'for', 'frock', 'gown',
       'henley', 'hoodi', 'jacket', 'jean', 'jogger', 'jumpsuit',
       'kaftan', 'kimono', 'knit', 'leg', 'maxi', 'midi', 'mini',
       'overal', 'overcoat', 'pant', 'parka', 'peacoat', 'poncho',
       'print', 'pullov', 'romper', 'sheath', 'shift', 'shirt',
       'shirtdress', 'skirt', 'skort', 'suit', 'sweater', 'sweatershirt',
       'sweatpant', 'sweatshirt', 't-shirt', 'tank', 'tee', 'tight',
       'top', 'trench', 'trouser', 'tunic', 'turtleneck', 'vest'],
      dtype='<U12')

In [99]:
def category_identity (category):
    category = stemmer.stem(category)
    long = ['ballgown','caftan','cape','combo','dress','duster','frock','for','gown','jumpsuit','kaftan','kimono','maxi'
           ,'midi','mini','overal','overcoat','print','romper','sheath','shirtdress']
    bot = ['culott','down','jean','jogger','leg','pant','sweatpant','trouser']
    top_long = ['blazer','blouson','bomber','buttondown','cardigan','coat','crewneck','henley','hoodi','jacket','knit',
                'parka','peacoat','poncho','pullov','suit','sweater','sweatershirt','sweatshirt','trench','tunic','turtleneck']
    top_short = ['blous','cami','shift','shirt','skirt','skort','t-shirt','tank','tee','tight','top','vest','over','blou']
    if category in long:
        return 'long'
    if category in bot:
        return 'bot'
    if category in top_long:
        return 'top_long'
    if category in top_short:
        return 'top_short' 
    return category

In [100]:
data_tb['category'] = data_tb.category.apply(category_identity)

In [101]:
np.unique(([category_identity(d) for d in category_stem]))

array(['bot', 'long', 'top_long', 'top_short'], dtype='<U9')

## update the weight

In [102]:
#convert to weight to float then minus the average of weight
def convert_weight(strin):
    if isinstance(strin, str):
        return (float(strin.strip('lbs')) -137) *0.4535
    else:
        return (float(strin) - 137) *0.4535
    
#convert to height to cm then minus the average of height
def convert_height(strin):
    if isinstance(strin, str):
        #print(strin)
        fet, inc = strin.split()
        fet = float(fet.strip("'").strip('"'))
        inc = float(inc.strip("'").strip('"'))
        return (fet * 30.48+ inc * 2.54) - 165.89

def normalize_the_size(strin):
    return strin - 12.245175128801728
#fillna with the average - average, which is 0
data_tb.weight = data_tb.weight.apply(convert_weight).fillna(0)
data_tb.height = data_tb.height.apply(convert_height).fillna(0)

#change the size to float
data_tb['size'] = data_tb['size'].astype(float).apply(normalize_the_size)

In [103]:
data_tb.height.std()

6.753335257526709

In [104]:
data_tb.weight.std()

9.125903724116197

In [105]:
data_tb['size'].std()

8.494876687232058

since we think the height people tend to have a higher weight and bigger size of dress, so we add those three value together.

In [106]:
data_tb['bust size'] = data_tb['bust size'].fillna('36c')

In [107]:
def get_bust_num (data):
    return int(data[:2])
def get_bust_character(data):
    return data[2:]

In [108]:
data_tb['bust_num'] = data_tb['bust size'].apply(get_bust_num)
data_tb['bust_char'] = data_tb['bust size'].apply(get_bust_character)

In [109]:
data_tb['body type'] = data_tb['body type'].fillna('other')

In [110]:
data_tb['rented for'] = data_tb['rented for'].fillna('other')

In [111]:
data_tb.age = data_tb.age.astype('float')

In [112]:
data_tb.age = data_tb.age.fillna(data_tb.age.mean())
data_tb.age = np.log(data_tb.age + 1)

In [113]:
data_tb.rating = data_tb.rating.astype('float')
data_tb.rating = data_tb.rating.fillna(data_tb.rating.mean())

In [114]:
data_tb

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date,bust_num,bust_char
0,fit,420272,34d,2260466,0.0000,10.0,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,long,6.83,1.754825,3.367296,"April 20, 2016",34,d
1,fit,273551,34b,153475,-2.2675,10.0,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,long,1.75,-0.245175,3.610918,"June 18, 2013",34,b
2,fit,360448,36c,1063761,0.0000,10.0,party,This hugged in all the right places! It was a ...,other,It was a great time to celebrate the (almost) ...,long,-3.33,-8.245175,4.762174,"December 14, 2015",36,c
3,fit,909926,34c,126335,-0.9070,8.0,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,long,-0.79,-4.245175,3.555348,"February 12, 2014",34,c
4,fit,151944,34b,616682,3.6280,10.0,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,long,9.37,-0.245175,3.332205,"September 26, 2016",34,b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192539,fit,66386,34dd,2252812,1.3605,10.0,work,Fit like a glove!,hourglass,LOVE IT!!! First Item Im thinking of buying!,long,9.37,-4.245175,3.761200,"May 18, 2016",34,dd
192540,fit,118398,32c,682043,-16.7795,10.0,work,The pattern contrast on this dress is really s...,petite,LOVE it!,long,-10.95,-8.245175,3.401197,"September 30, 2016",32,c
192541,fit,47002,36a,683251,-0.9070,6.0,everyday,"Like the other DVF wraps, the fit on this is f...",straight & narrow,"Loud patterning, flattering fit",long,6.83,-4.245175,3.465736,"March 4, 2016",36,a
192542,fit,961120,36c,126335,12.6980,10.0,wedding,This dress was PERFECTION. it looked incredib...,pear,loved this dress it was comfortable and photog...,long,1.75,3.754825,3.465736,"November 25, 2015",36,c


In [115]:
data_tb = data_tb.drop(['bust size'], axis=1)

In [116]:
data_tb.isnull().sum()

fit               0
user_id           0
item_id           0
weight            0
rating            0
rented for        0
review_text       0
body type         0
review_summary    0
category          0
height            0
size              0
age               0
review_date       0
bust_num          0
bust_char         0
dtype: int64

# Save to csv

In [117]:
data_tb.to_csv('clean.csv', index=False)

In [104]:
cv = TfidfVectorizer(stop_words = 'english',ngram_range=(1,1),sublinear_tf = True)
x_text_train = [d['review_text'] for d in train_set]
x_train_cv = cv.fit_transform(x_text_train)

In [105]:
x_train_cv

<173289x26907 sparse matrix of type '<class 'numpy.float64'>'
	with 3917579 stored elements in Compressed Sparse Row format>

Moreover, there is still has some repeated categories such as t_shirt and tee, blous and blouson, sweater, sweatershirt and sweatshirt. Therefore, we need to clear the category type futher

In [120]:
x_text_train = [d['review_text'] for d in train_set]
y_text_train = [d['category'] for d in train_set]
x_text_test = [d['review_text'] for d in validation_set]
y_text_test = [d['category'] for d in validation_set]